# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p> 
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [1]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login
login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

In [2]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-OpenO1-SFT"
finetune_tags = ["smol-course", "module_1"]

# Generate with the base model

Here we will try out the base model which does not have a chat template. 

In [3]:
# Let's test the base model before training
prompt = "Write a Python program that adds two integers"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a Python program that adds two integers
Write a Python program that prints the sum of two integers
Write a Python program that prints the sum of two integers
Write a Python program that prints the sum of two integers
Write a Python program that prints the sum of two integers
Write a Python program that prints the sum of two integers
Write a Python program that prints the sum of two integers
Write a Python program that prints the sum of two integers
Write a Python program that prints the sum of two integers
Write a Python program


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [4]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset("O1-OPEN/OpenO1-SFT")

In [5]:
print(ds["train"][1])

{'instruction': 'Create a Python class that encodes a given number using the Full Kociołek Encryption algorithm. The algorithm should involve multiple stages of mathematical operations and conditional logic. Ensure that the class methods handle all possible edge cases, such as negative numbers, fractions, and large values. Additionally, add a method to decode a number that has been previously encoded using the same algorithm. The decoding method should reverse the mathematical operations performed during encoding and return the original number. The class should also include error handling to handle invalid input and a method to generate a random encryption key for each instance of the class. Finally, write a series of test cases to verify the correctness of the class implementation, covering different types of input and edge cases.', 'output': '<Thought>\nAlright, I need to create a Python class that implements the Full Kociołek Encryption algorithm. First, I should understand what thi

In [9]:
ds = load_dataset("O1-OPEN/OpenO1-SFT")


ds = ds["train"].train_test_split(test_size=0.05, seed=42)

def process_dataset(sample):
    messages = [
        {"role": "user", "content": sample["instruction"]},
        {"role": "assistant", "content": sample["output"]}
    ]
    sample["chatml"] = tokenizer.apply_chat_template(messages, tokenize=False)

    return sample


ds = ds.map(process_dataset)
# TODO: 🦁 If your dataset is not in a format that TRL can convert to the chat template, you will need to process it. Refer to the [module](../chat_templates.md)

Map:   0%|          | 0/73800 [00:00<?, ? examples/s]

Map:   0%|          | 0/3885 [00:00<?, ? examples/s]

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [10]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'chatml'],
        num_rows: 73800
    })
    test: Dataset({
        features: ['instruction', 'output', 'chatml'],
        num_rows: 3885
    })
})


In [17]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=1,  # Set according to your GPU memory capacity
    gradient_accumulation_steps=1,
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
    dataset_text_field="chatml",
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

/home/user/HuaProjects/Tookies/smol-course/.venv/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/user/HuaProjects/Tookies/smol-course/.venv/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [18]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

  0%|          | 0/1000 [00:00<?, ?it/s]

{'loss': 1.689, 'grad_norm': 4.789322376251221, 'learning_rate': 4.9500000000000004e-05, 'epoch': 0.0}
{'loss': 1.6809, 'grad_norm': 2.5645437240600586, 'learning_rate': 4.9e-05, 'epoch': 0.0}
{'loss': 1.4344, 'grad_norm': 1.8341213464736938, 'learning_rate': 4.85e-05, 'epoch': 0.0}
{'loss': 1.6071, 'grad_norm': 2.0909006595611572, 'learning_rate': 4.8e-05, 'epoch': 0.0}
{'loss': 1.3541, 'grad_norm': 2.3180649280548096, 'learning_rate': 4.75e-05, 'epoch': 0.0}


  0%|          | 0/486 [00:00<?, ?it/s]

{'eval_loss': 1.453147530555725, 'eval_runtime': 92.1703, 'eval_samples_per_second': 42.15, 'eval_steps_per_second': 5.273, 'epoch': 0.0}
{'loss': 1.4948, 'grad_norm': 2.5666050910949707, 'learning_rate': 4.7e-05, 'epoch': 0.0}
{'loss': 1.591, 'grad_norm': 2.4415390491485596, 'learning_rate': 4.6500000000000005e-05, 'epoch': 0.0}
{'loss': 1.5125, 'grad_norm': 2.3992252349853516, 'learning_rate': 4.600000000000001e-05, 'epoch': 0.0}
{'loss': 1.4225, 'grad_norm': 2.355292320251465, 'learning_rate': 4.55e-05, 'epoch': 0.0}
{'loss': 1.4036, 'grad_norm': 2.748029947280884, 'learning_rate': 4.5e-05, 'epoch': 0.0}


  0%|          | 0/486 [00:00<?, ?it/s]

...

{'eval_loss': 1.346459150314331, 'eval_runtime': 93.8329, 'eval_samples_per_second': 41.403, 'eval_steps_per_second': 5.179, 'epoch': 0.01}
{'loss': 1.344, 'grad_norm': 2.1930482387542725, 'learning_rate': 4.5e-06, 'epoch': 0.01}
{'loss': 1.2041, 'grad_norm': 1.9583771228790283, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.01}
{'loss': 1.604, 'grad_norm': 2.241290807723999, 'learning_rate': 3.5000000000000004e-06, 'epoch': 0.01}
{'loss': 1.2927, 'grad_norm': 2.1759748458862305, 'learning_rate': 3e-06, 'epoch': 0.01}
{'loss': 1.4155, 'grad_norm': 1.7820384502410889, 'learning_rate': 2.5e-06, 'epoch': 0.01}


  0%|          | 0/486 [00:00<?, ?it/s]

{'eval_loss': 1.345608115196228, 'eval_runtime': 93.2482, 'eval_samples_per_second': 41.663, 'eval_steps_per_second': 5.212, 'epoch': 0.01}
{'loss': 1.2435, 'grad_norm': 2.1484668254852295, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}
{'loss': 1.5263, 'grad_norm': 1.6413675546646118, 'learning_rate': 1.5e-06, 'epoch': 0.01}
{'loss': 1.3848, 'grad_norm': 2.2584738731384277, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}
{'loss': 1.3049, 'grad_norm': 2.09061336517334, 'learning_rate': 5.000000000000001e-07, 'epoch': 0.01}
{'loss': 1.333, 'grad_norm': 2.1181726455688477, 'learning_rate': 0.0, 'epoch': 0.01}


  0%|          | 0/486 [00:00<?, ?it/s]

{'eval_loss': 1.3451868295669556, 'eval_runtime': 93.4894, 'eval_samples_per_second': 41.555, 'eval_steps_per_second': 5.198, 'epoch': 0.01}
{'train_runtime': 1966.3726, 'train_samples_per_second': 0.509, 'train_steps_per_second': 0.509, 'train_loss': 1.3622560482025146, 'epoch': 0.01}


In [19]:
trainer.push_to_hub(tags=finetune_tags)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Tookies/SmolLM2-FT-OpenO1-SFT/commit/b9b0bc9d1581565ea390a7940405b211d7de1453', commit_message='End of training', commit_description='', oid='b9b0bc9d1581565ea390a7940405b211d7de1453', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Tookies/SmolLM2-FT-OpenO1-SFT', endpoint='https://huggingface.co', repo_type='model', repo_id='Tookies/SmolLM2-FT-OpenO1-SFT'), pr_revision=None, pr_num=None)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [23]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
prompt = "Write a Python program that adds two integers"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: use the fine-tuned to model generate a response, just like with the base example.
outputs = model.generate(**inputs, max_new_tokens=1500)
print("After training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

After training:
user
Write a Python program that adds two integers
assistant
<Thought>
Alright, I need to write a Python program that adds two integers. Let's break this down step by step.

First, I should consider the input format. The program should accept two integers as input. The first integer should be the first integer, and the second integer should be the second integer.

Next, I need to handle the comparison between the two integers. If the first integer is greater than the second, I should return `True`. If the first integer is less than the second, I should return `False`.

I should also consider edge cases. For example, what if the first integer is zero? In that case, the program should return `False`. What if the first integer is negative? In that case, the program should return `False`. What if the first integer is zero and the second integer is negative? In that case, the program should return `False`.

Also, I need to handle the case where the first integer is zero and 

In [24]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: use the fine-tuned to model generate a response, just like with the base example.
outputs = model.generate(**inputs, max_new_tokens=500)
print("After training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

After training:
user
Write a haiku about programming
assistant
<Thought>
Alright, I need to write a haiku about programming. Let's start by understanding what programming is. Programming is the process of writing code to solve problems or perform tasks. It involves using programming languages like Python, Java, or C++ to create software applications.

Now, programming is a broad field, and there are many different programming languages. Python is a popular choice because of its simplicity and powerful features. Java is another option, but I'll stick with Python for simplicity.

So, I'll need to choose a programming language that's easy to learn and understand. Python is a good choice because it's a general-purpose programming language that's easy to learn and use.

Next, I'll think about the programming language's structure. Python is a dynamically typed language, which means it's not tied to a specific type of data. This makes it more flexible and adaptable.

I'll also consider the pr

In [ ]:
from IPython.display import display, HTML
display(HTML("<h2>Kernel will shut down...</h2>"))

import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True) 

{'status': 'ok', 'restart': True}

: 

## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.